<a href="https://colab.research.google.com/github/ferdouszislam/pytorch-practice/blob/main/cross_entropy_from_label_probabilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch
import torch.nn as nn

In [21]:
# Cross Entropy Loss implementation was modified from- <https://discuss.pytorch.org/t/how-to-write-custom-crossentropyloss/58072> 

def log_softmax(x):
  return x - torch.logsumexp(x,dim=1, keepdim=True)

def cross_entropy(outputs, targets):
  num_examples = targets.shape[0]
  cross_entropies = targets*log_softmax(outputs)
  return - torch.sum(cross_entropies) / num_examples

In [22]:
outputs = torch.tensor([[300.0, 100.0, 400.0, 100.0, 100.0],
                        [100.0, 200.0, 300.0, 200.0, 200.0],
                        [300.0, 100.0, 400.0, 100.0, 100.0]])

labels = torch.tensor([3, 2, 1])


label_one_hot = torch.tensor([[0, 0, 0, 1, 0], 
                              [0, 0, 1, 0, 0], 
                              [0, 1, 0, 0, 0]])

label_probs = torch.tensor([[0.1, 0.2, 0.1, 0.4, 0.2], 
                            [0.2, 0.2, 0.4, 0.1, 0.1], 
                            [0.1, 0.6, 0.1, 0.1, 0.1]])

In [23]:
# get labels from probs
_, labels_from_label_probs = torch.max(label_probs, 1)
labels_from_label_probs

tensor([3, 2, 1])

In [24]:
built_in_ce = nn.CrossEntropyLoss()
built_in_ce(outputs, labels)

tensor(200.)

In [25]:
cross_entropy(outputs, label_one_hot)

tensor(200.)

In [26]:
cross_entropy(outputs, label_probs)

tensor(193.3333)